In [1]:
import re
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df_all = pd.read_pickle('./data/df_all_lemma.pkl')
print(len(df_all))
df_all.head(1)

2467


,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views,transcript
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110,it been great hasnt it. ive been blown away by...


In [3]:
docs_all = list(df_all['transcript'])

from nltk.corpus import stopwords

stops_standard = stopwords.words('english')
stops_custom = ['shes','youll','ill','yeah','th','yes','oh',
                'ok','okay','might','ha','mr','bg','ms',
                'mrs','ca','em','da','ted']+[re.sub('[^A-Za-z ]+', '', w) for w in stops_standard]
stop_list = list(set(stops_standard + stops_custom))

print(stop_list)

['the', 'only', 'oh', 'its', 'in', 'those', 'aren', 'needn', 'ma', "won't", 'does', 'herself', 'once', "it's", 'ok', 'shes', 'wasnt', 'is', 'hadn', 'ca', 'havent', 'okay', 'between', 'can', 'youll', 'neednt', 'ours', 'that', "couldn't", 'won', 'now', 'his', 'should', 'yourselves', 'your', 'so', 'had', "wouldn't", 'ill', 'myself', "isn't", 'itself', 'again', 'shouldn', 'mightnt', 'but', 'will', 'under', 'not', 'thatll', 'are', 'over', 'i', 'mr', 'did', 'further', 'being', "don't", "haven't", 'shant', 'a', 'ourselves', "you're", 'youre', 'wouldnt', 'above', "wasn't", 'whom', 'himself', 'into', 'shan', 'yourself', 'to', 'we', 'while', 's', 'at', 'isn', 'same', 'arent', 'until', 'up', 'doesn', 'haven', 'was', 'how', 'wasn', 'if', 'about', 'below', 'any', 'out', 'having', 'such', 'than', 'her', 'by', "mustn't", 'and', 'themselves', 'or', 'on', 'youve', 'th', 'ted', 'from', 'each', "hasn't", "shouldn't", 'were', 'all', 've', 'yeah', 'bg', 'against', 'down', 'em', 'dont', 'for', 'd', "hadn't"

In [13]:
docs = docs_all

from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

n_samples = 2000
n_features = 2000
n_components = 50
n_top_words = 10
n_gram = 1
alpha = 0.1
stop_choice= stop_list

max_df = 400
min_df = 10
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()


# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=max_df, min_df=min_df,
                                ngram_range=(n_gram,n_gram),
                                max_features=n_features,
                                stop_words=stop_choice)
t0 = time()
tf = tf_vectorizer.fit_transform(docs)
print("done in %0.3fs." % (time() - t0))
print()

# Fit the LDA model
print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Extracting tf features for LDA...
done in 2.623s.

Fitting LDA models with tf features, n_samples=2000 and n_features=2000...
done in 10.931s.

Topics in LDA model:
Topic #0: cancer blood patient tumor tissue surgery breast organ stem lung
Topic #1: military fly department flight strategy capability russian intelligence moon boy
Topic #2: ocean fish sea coral shark boat whale reef underwater island
Topic #3: religion sugar religious moral dinosaur silk spider muslim tradition christian
Topic #4: music stick listen gap please etc instrument musician tiny hall
Topic #5: ah wow radio slide produce wind library crop dropped blow
Topic #6: bread cow milk spread remarkable hot paint ad ingredient sell
Topic #7: la boy de poem chose fresh busy written bringing curious
Topic #8: online photo google paper page fun algorithm conversation card box
Topic #9: evolution theory africa laptop memory prediction organism wish evolve digital
Topic #10: cave mountain hidden ride continent mar covered expl

In [14]:
X = lda.fit_transform(tf)


In [15]:
X[134].argmax()

15

### Find related talks

In [7]:
import sklearn.neighbors.kd_tree as kdtree

In [16]:
tree = kdtree.KDTree(X, leaf_size=10) 
N = 1000
dist, ind = tree.query([X[N]], k=20)  
for i in ind[0]:
    print(i,'\t',df_all.iloc[i].title)
#     print(i,'\t',df_all.iloc[i].views)
#     print(df_all.iloc[i].tags)

1000 	 A prosthetic arm that "feels"
2146 	 This scientist makes ears out of apples
1787 	 This gel can make you stop bleeding instantly
1554 	 Body parts on a chip
1126 	 On the virtual dissection table
2400 	 Lifesaving scientific tools made of paper
1177 	 What we didn't know about penis anatomy
2195 	 Why helmets don't prevent concussions -- and what might
859 	 Human exoskeletons -- for war and healing
228 	 Moving sculpture
248 	 Animate characters by evolving them
1378 	 Play with smart materials
589 	 A lab the size of a postage stamp
815 	 Visualizing the medical data explosion
1566 	 The $80 prosthetic knee that's changing lives
1013 	 High-tech art (with a sense of humor)
668 	 Pointing to the future of UI
705 	 The bio-future of joint replacement
1629 	 The new bionics that let us run, climb and dance
938 	 Caring for engineered tissue


In [17]:
df_related = pd.DataFrame(eval(df_all.iloc[N]['related_talks']))
for k in range(len(df_related)):
    print(list(df_all[df_all['title']==df_related.iloc[k]['title']].title))
#     print(df_all[df_all['title']==df_related.iloc[k]['title']].views)
#     print(list(df_all[df_all['title']==df_related.iloc[k]['title']].tags))

['My 12 pairs of legs']
['Luke, a new prosthetic arm for soldiers']
['The potential of regenerative medicine']
['The mystery of chronic pain']
['The emotion behind invention']
['3 clues to understanding your brain']


In [12]:
# print(docs_all[395])

### Clustering using KMeans

In [509]:
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn.metrics import silhouette_score
from sklearn import cluster

n_clusters = 50

# ward = cluster.AgglomerativeClustering(
#     n_clusters=n_clusters, linkage='ward',
#     connectivity=connectivity)
spectral = cluster.SpectralClustering(
    n_clusters=n_clusters, eigen_solver='arpack',
    affinity="nearest_neighbors")
Xp = spectral.fit_predict(X)
# dbscan = cluster.DBSCAN(eps=params['eps'])

# km = KMeans(n_clusters=50)
# Xt = km.fit_transform(X)

# mu_docs = km.cluster_centers_
# plt.plot(mu_docs.transpose())

# silhouette_score(X, km.labels_, metric='euclidean')

# sil_scores = []
# iner_scores = []
# ks = range(2,20)
# for k in ks:
#     km = KMeans(n_clusters=k)
#     km.fit(X)
#     sil_scores.append(silhouette_score(X, km.labels_, metric='euclidean'))
#     iner_scores.append(km.inertia_)
    
# plt.plot(ks,sil_scores)
# plt.plot(ks,iner_scores)

In [510]:
Xp.

array([18, 29, 18, ..., 28, 18, 10], dtype=int32)

In [505]:
tree = kdtree.KDTree(Xt, leaf_size=10) 
# N = 1346
dist, ind = tree.query([Xt[N]], k=10)  
for i in ind[0]:
    print(i,'\t',df_all.iloc[i].title)
#     print(i,'\t',df_all.iloc[i].views)
#     print(df_all.iloc[i].tags)

1599 	 Love -- you're doing it wrong
2343 	 3 ways to plan for the (very) long term
2085 	 How to get back to work after a career break
1395 	 What makes us feel good about our work?
1135 	 How I beat a patent troll
1872 	 How equal do we want the world to be? You'd be surprised
1821 	 The danger of hiding who you are
1870 	 5 ways to kill your dreams
1566 	 The $80 prosthetic knee that's changing lives
609 	 Plug into your hard-wired happiness


In [506]:
df_all.iloc[2343].transcript

'and theyd say great. and ive been seeing that time horizon get shorter and shorter and shorter so much so that i met with a ceo two month ago and i said we started our initial conversation. he go i love what you do. i want to talk about the next six month. we have a lot of problem that we are facing. these are civilizationalscale problem. the issue though is we cant solve them using the mental model that we use right now to try and solve these problem. yes a lot of great technical work is being done but there is a problem that we need to solve for a priori before if we want to really move the needle on those big problem. shorttermism. right. there no march. there no bracelet. there no petition that you can sign to be against shorttermism. i tried to put one up and no one signed. it wa weird. but it prevents u from doing so much. shorttermism for many reason ha pervaded every nook and cranny of our reality. i just want you to take a second and just think about an issue that youre think